In [ ]:
import pandas as pd


In [25]:
base = pd.read_excel("../Bases/Base_tratada.xlsx")

In [33]:
import numpy as np  # Adicionado para usar np.log2
from scipy.stats import entropy

def entropia_padrao(column, normalize=False):
    prob = column.value_counts(normalize=True)
    ent = entropy(prob, base=2)
    if normalize:
        # Normaliza pela entropia máxima possível (log2(n) onde n é o número de categorias)
        n_categories = len(prob)
        if n_categories <= 1:
            return 0.0  # Evita divisão por zero se houver apenas uma categoria
        max_entropy = np.log2(n_categories)
        return ent / max_entropy if max_entropy > 0 else 0.0
    return ent

def entropia_condicional(atributo, classe, normalize=False):
    df = pd.concat([atributo, classe], axis=1)
    valores = atributo.unique()
    total = len(atributo)
    entropia_cond = 0.0
    for valor in valores:
        subset = df[df[atributo.name] == valor][classe.name]
        proporcao = len(subset) / total
        entropia_subset = entropia_padrao(subset, normalize)
        entropia_cond += proporcao * entropia_subset
    return entropia_cond

# Exemplo de uso
classe = 'Q074'  # substitua pelo nome da sua coluna classe
entropia_classe = entropia_padrao(base[classe])
print(f"Entropia da classe: {round(entropia_classe,4)}")
print("-" * 95)

ganhos = {}
normalize = True  # DEFINA AQUI SE QUER NORMALIZAR (True) OU NÃO (False)

print(f"{'Atributo':<30} {'Entropia Padrão':>18} {'Entropia Condicional':>22} {'Ganho de Informação':>22}")
print("-" * 95)

for atributo in base.columns:
    if atributo == classe:
        continue
    entropia_atr = entropia_padrao(base[atributo], normalize)
    entropia_cond = entropia_condicional(base[atributo], base[classe], normalize)
    ganho = entropia_classe - entropia_cond
    ganhos[atributo] = ganho
    print(f"{atributo:<30} {entropia_atr:>18.4f} {entropia_cond:>22.4f} {ganho:>22.4f}")

ganhos_ordenados = sorted(ganhos.items(), key=lambda x: x[1], reverse=True)

# Cabeçalho da tabela de atributos ordenados
print("\nAtributos ordenados por ganho de informação:")
print("-" * 51)
print(f"{'Atributo':<35} {'Ganho de Informação':>15}")
print("-" * 51)

# Corpo da tabela
for atributo, ganho in ganhos_ordenados:
    # Formatando com 4 casas decimais e alinhamento correto
    print(f"{atributo[:32]:<35} {ganho:>15.4f}")  # Limita o nome do atributo a 32 caracteres


Entropia da classe: 0.3604
-----------------------------------------------------------------------------------------------
Atributo                          Entropia Padrão   Entropia Condicional    Ganho de Informação
-----------------------------------------------------------------------------------------------
V0001                                      0.9390                 0.3597                 0.0007
V0022                                      0.9221                 0.3601                 0.0003
A02201|A02305|A02306                       0.8268                 0.3599                 0.0005
C006                                       0.9968                 0.3587                 0.0017
C008                                       0.8828                 0.3600                 0.0004
C009                                       0.9472                 0.3603                 0.0001
N010                                       0.8238                 0.3583                 0.0021
P00104       